In [202]:
import fiona
import pandas as pd
import geopandas
import os
import numpy as np
import shapely.wkt

In [203]:
os.chdir('/Users/dannyha/SoCalEVCSNetwork')
os.getcwd()

'/Users/dannyha/SoCalEVCSNetwork'

# Data Importing/Cleaning

## SoCal Zipcodes

In [204]:
socal_zipcodes = pd.read_csv (r'data/socal_zipcodes/socal_zip.csv')
socal_zipcodes = socal_zipcodes[['ZCTA5CE10','GEOID10','INTPTLAT10','INTPTLON10','geometry']]
socal_zipcodes = socal_zipcodes.rename(columns={"ZCTA5CE10": "Zip Code", "GEOID10": "GEOID", "INTPTLAT10": "Latitude",
                              "INTPTLON10": "Longitude", "geometry": "Geometry"})
socal_zipcodes['Zip Code'] = socal_zipcodes['Zip Code'].apply(str)

In [205]:
socal_zipcodes.head()

,Zip Code,GEOID,Latitude,Longitude,Geometry
0,91504,691504,34.204568,-118.326365,"POLYGON ((-118.348583 34.208461, -118.348564 3..."
1,91606,691606,34.185767,-118.388324,"POLYGON ((-118.403066 34.19394, -118.402564 34..."
2,90640,690640,34.015444,-118.111012,"POLYGON ((-118.128344 33.985893, -118.127359 3..."
3,91770,691770,34.065493,-118.084529,"POLYGON ((-118.107291 34.066222, -118.107323 3..."
4,90290,690290,34.107728,-118.615981,"POLYGON ((-118.640641 34.125821, -118.640133 3..."


## LEHD Data

In [249]:
#column formatting
commuting = pd.read_csv (r'data/commuting/ca_od_aux_JT00_2018.csv')
commuting.columns = ['Workplace Census Block Code', 'Residence Census Block Code',
                    'Total number of jobs', 'Number of jobs of workers age 29 or younger',
                    'Number of jobs for workers age 30 to 54', 'Number of jobs for workers age 55 or older',
                    'Number of jobs with earnings $1250/month or less',
                     'Number of jobs with earnings $1251/month to $3333/month',
                    'Number of jobs with earnings greater than $3333/month',
                    'SI01','SI02','SI01','date']
commuting_filtered = commuting[['Workplace Census Block Code', 'Residence Census Block Code',
                    'Total number of jobs']]
commuting_filtered = commuting_filtered.reset_index().drop(columns=['index'])

#converting census codes to strings and trimming
commuting_filtered['Workplace Census Block Code'] = commuting_filtered['Workplace Census Block Code'].apply(str)
commuting_filtered['Residence Census Block Code'] = commuting_filtered['Residence Census Block Code'].apply(str)
commuting_filtered['Workplace Census Block Code'] = commuting_filtered['Workplace Census Block Code'].str.replace(
    r'.{4}$', '')
commuting_filtered['Residence Census Block Code'] = commuting_filtered['Residence Census Block Code'].str.replace(
    r'.{4}$', '')

#renaming column to reflect code change
commuting_filtered = commuting_filtered.rename(
    columns={'Workplace Census Block Code':'Workplace Census Tract',
             'Residence Census Block Code':'Residence Census Tract',
            'Total number of jobs':'Job Count'})

In [250]:
#import tract to zip table to convert LEHD data to Zip Code
tract_to_zip = pd.read_csv (r'data/ZIP_TRACT_032010.csv')

tract_to_zip['ZIP'] = tract_to_zip['ZIP'].apply(str)

#filtering down to socal zipcode
socal_zips = tract_to_zip[tract_to_zip['ZIP'].isin(socal_zipcodes['Zip Code'])]
socal_zips['TRACT'] = socal_zips['TRACT'].apply(str)

#commuting_filtered_socal = commuting_filtered[commuting_filtered['Residence Census Tract'].isin(socal_zips['TRACT'])]
commuting_filtered_socal = commuting_filtered[commuting_filtered['Workplace Census Tract'].isin(socal_zips['TRACT'])]
commuting_filtered_socal

<ipython-input-250-781278b6dfbb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  socal_zips['TRACT'] = socal_zips['TRACT'].apply(str)


,Workplace Census Tract,Residence Census Tract,Job Count
27391,6037101210,4001944901,1
27392,6037101210,4013104202,1
27393,6037101220,51770000601,1
27394,6037101220,12099004815,1
27395,6037101220,17031801701,1
...,...,...,...
96501,6059110606,37007920100,1
96502,6059110606,37125951100,1
96503,6059110607,4013061041,1
96504,6059110607,4013082205,1


In [251]:
commuting_filtered_socal.columns=['TRACT','Origin Tract','Jobs']
commuting_filtered_socal = commuting_filtered_socal.reset_index()
commuting_filtered_socal = commuting_filtered_socal.drop(columns=['index'])

In [252]:
#cleaning
socal_tract_to_zip = tract_to_zip[tract_to_zip['ZIP'].isin(socal_zipcodes['Zip Code'])].reset_index()
socal_tract_to_zip['TRACT'] = socal_tract_to_zip['TRACT'].apply(str)
socal_tract_to_zip = socal_tract_to_zip[['ZIP','TRACT']]
socal_tract_to_zip

,ZIP,TRACT
0,90001,6037239200
1,90001,6037239330
2,90001,6037239500
3,90001,6037239800
4,90001,6037240010
...,...,...
4091,93552,6037910901
4092,93553,6037910000
4093,93553,6037910901
4094,93553,6037910902


In [253]:
merged = commuting_filtered_socal.merge(socal_tract_to_zip, on='TRACT')
merged.columns = ['Destination Tract','Origin Tract','Jobs','Destination Zipcode']
merged

,Destination Tract,Origin Tract,Jobs,Destination Zipcode
0,6037101210,4001944901,1,91042
1,6037101210,4013104202,1,91042
2,6037101220,51770000601,1,91042
3,6037101220,12099004815,1,91042
4,6037101220,17031801701,1,91042
...,...,...,...,...
95079,6059110606,37007920100,1,90621
95080,6059110606,37125951100,1,90621
95081,6059110607,4013061041,1,90621
95082,6059110607,4013082205,1,90621


### Grouped by Destination Tracts

In [254]:
#conducting groupby on merged destination zipcode table- these are heavy destination tracts
merged_groupby_zip = merged[['Destination Zipcode','Jobs']].groupby('Destination Zipcode').sum()
merged_groupby_zip.sort_values('Jobs',ascending=False)

,Jobs
Destination Zipcode,
91505,3855
92705,3343
92707,3246
92626,2578
92806,2407
...,...
90067,4
92662,4
90293,3


In [255]:
merged_groupby_zip_2 = merged[['Destination Zipcode','Origin Tract','Jobs']].groupby(
    ['Destination Zipcode','Origin Tract']).sum()
merged_groupby_zip_2.sort_values('Jobs',ascending=False)

Jobs
Destination Zipcode Origin Tract      
91505               36061015700     13
                    36061017900     13
                    36061015900     13
                    36061016900     12
                    36061013900     12
...                                ...
90746               4013216852       1
                    4013216849       1
                    4013216846       1
                    4013216841       1
93552               48373210301      1

[87884 rows x 1 columns]

In [256]:
merged_groupby_zip_shape = merged_groupby_zip.reset_index()
merged_groupby_zip_shape.columns = ['Zip Code','Jobs']

In [257]:
socal_zipcodes_geom = socal_zipcodes[['Zip Code','Geometry']]

In [258]:
merged_groupby_zip_shape = merged_groupby_zip_shape.merge(socal_zipcodes_geom,on='Zip Code')
merged_groupby_zip_shape

,Zip Code,Jobs,Geometry
0,90001,115,"POLYGON ((-118.237949 33.96015, -118.238527 33..."
1,90002,36,"POLYGON ((-118.23737 33.958521, -118.237377 33..."
2,90003,101,"POLYGON ((-118.282698 33.964166, -118.282698 3..."
3,90004,85,"POLYGON ((-118.311601 34.06895799999999, -118...."
4,90005,142,"MULTIPOLYGON (((-118.302852 34.062356, -118.30..."
...,...,...,...
342,93510,14,"POLYGON ((-118.283479 34.492036, -118.282961 3..."
343,93543,11,"POLYGON ((-117.938869 34.470802, -117.939103 3..."
344,93550,91,"POLYGON ((-118.189699 34.37524, -118.189689 34..."
345,93551,90,"POLYGON ((-118.201212 34.630971, -118.199343 3..."


In [259]:
save_geom = []

for i in merged_groupby_zip_shape['Geometry'].tolist():
    save_geom.append(shapely.wkt.loads(i))

merged_groupby_zip_shape['geometry'] = save_geom
#merged_groupby_zip_shape = merged_groupby_zip_shape.drop(columns=['Geometry'])
    
merged_groupby_zip_shape

,Zip Code,Jobs,Geometry,geometry
0,90001,115,"POLYGON ((-118.237949 33.96015, -118.238527 33...","POLYGON ((-118.237949 33.96015, -118.238527 33..."
1,90002,36,"POLYGON ((-118.23737 33.958521, -118.237377 33...","POLYGON ((-118.23737 33.958521, -118.237377 33..."
2,90003,101,"POLYGON ((-118.282698 33.964166, -118.282698 3...","POLYGON ((-118.282698 33.964166, -118.282698 3..."
3,90004,85,"POLYGON ((-118.311601 34.06895799999999, -118....","POLYGON ((-118.311601 34.06895799999999, -118...."
4,90005,142,"MULTIPOLYGON (((-118.302852 34.062356, -118.30...","(POLYGON ((-118.302852 34.062356, -118.302852 ..."
...,...,...,...,...
342,93510,14,"POLYGON ((-118.283479 34.492036, -118.282961 3...","POLYGON ((-118.283479 34.492036, -118.282961 3..."
343,93543,11,"POLYGON ((-117.938869 34.470802, -117.939103 3...","POLYGON ((-117.938869 34.470802, -117.939103 3..."
344,93550,91,"POLYGON ((-118.189699 34.37524, -118.189689 34...","POLYGON ((-118.189699 34.37524, -118.189689 34..."
345,93551,90,"POLYGON ((-118.201212 34.630971, -118.199343 3...","POLYGON ((-118.201212 34.630971, -118.199343 3..."


In [263]:
merged_groupby_zip_shape = geopandas.GeoDataFrame(merged_groupby_zip_shape, geometry='geometry')

In [264]:
#export
merged_groupby_zip_shape.to_file('merged_groupby_zip_shape.shp', driver='ESRI Shapefile')

### Grouped by Origin Tracts

In [118]:
commuting_filtered

,Workplace Census Tract,Residence Census Tract,Job Count
0,6001400100,25005645102,1
1,6001400100,13241970201,1
2,6001400200,48039661501,1
3,6001400200,41047001502,1
4,6001400200,45059921002,1
...,...,...,...
179250,6115041100,32031003507,1
179251,6115041100,5043490200,1
179252,6115041100,32005001000,1
179253,6115041100,32031002207,1


### Correlation with Charger Density

In [ ]:
chargers_socal_acs_kmeans_zipcodes.geojson